# Scalar Computation Graph v2

## Now support weights sharing!

In [1]:
import numpy as np
import abc


topo_list = []  # Topological sort

class Node(abc.ABC):
    
    def __init__(self):
        topo_list.append(self)
        self._dout = 0.0
    
    @abc.abstractmethod
    def forward(self):
        """Feed Forward"""
    
    @abc.abstractmethod
    def backward(self):
        """Back Propagate"""
    
    def as_terminal(self):
        """Let the node be the terminal """
        self._dout = 1.
        return self
    
    def propagte(self, dout):
        """Aggregate upstream gradients."""
        self._dout += dout

    @property
    def grads(self):
        return self._grads

In [2]:
class Variable(Node):
    
    def __init__(self, val):
        super().__init__()
        self._v = val
    
    def forward(self):
        return self._v
    
    def backward(self):
        self._grads = self._dout

        
class Add(Node):
    
    def __init__(self, a, b):
        super().__init__()
        self._a = a
        self._b = b
    
    def forward(self):
        v_a = self._a.forward()
        v_b = self._b.forward()
        return v_a + v_b
    
    def backward(self):
        dout = self._dout
        self._grads = [dout, dout]
        self._a.propagte(dout)
        self._b.propagte(dout)


class Mul(Node):
    
    def __init__(self, a, b):
        super().__init__()
        self._a = a
        self._b = b
    
    def forward(self):
        v_a = self._a.forward()
        v_b = self._b.forward()
        self._local_da = v_b
        self._local_db = v_a
        return v_a * v_b
    
    def backward(self):
        dout = self._dout
        da = self._local_da * dout
        db = self._local_db * dout
        self._grads = [da, db]
        self._a.propagte(da)
        self._b.propagte(db)
        

class Inv(Node):
    
    def __init__(self, a):
        super().__init__()
        self._a = a
    
    def forward(self):
        val = self._a.forward()
        self._local_grads = - 1. / val**2
        return 1. / val
    
    def backward(self):
        dout = self._dout
        self._grads = self._local_grads * dout
        self._a.propagte(self._grads)
        

class Exp(Node):
    
    def __init__(self, a):
        super().__init__()
        self._a = a
    
    def forward(self):
        val = self._a.forward()
        self._local_grads = np.exp(val)
        return self._local_grads
    
    def backward(self):
        dout = self._dout
        self._grads = self._local_grads * dout
        self._a.propagte(self._grads)

# Test

In [3]:
# init
topo_list = []
w1, w2, x1, x2, b = [Variable(float(i)) for i in [2, -3, -1, -2, -3]]

# build graph
logit = Add(Add(Mul(w1, x1), Mul(w2, x2)), b)
f = Inv(Add(Variable(1), Exp(Mul(logit, Variable(-1)))))

# forward
print('Forward:')
print(*('{:.2f}'.format(v.forward()) for v in topo_list))

# backward
f.as_terminal()
for v in reversed(topo_list):
    v.backward()
print('Backward:')
print(', '.join('{:.2f}'.format(v.grads) for v in [w1, w2, x1, x2, b]))

Forward:
2.00 -3.00 -1.00 -2.00 -3.00 -2.00 6.00 4.00 1.00 1.00 -1.00 -1.00 0.37 1.37 0.73
Backward:
-0.20, -0.39, 0.39, -0.59, 0.20


# Bug fixed!

In [4]:
topo_list = []

w, x1, x2 = [Variable(float(i)) for i in [5, 1, 2]]
f = Add(Mul(w, x1), Mul(w, x2))
print('Value: \n', f.forward())

f.as_terminal()
for v in reversed(topo_list):
    v.backward()
print('Gradients: \n', [v.grads for v in [w, x1, x2]])

Value: 
 15.0
Gradients: 
 [3.0, 5.0, 5.0]
